In [ ]:
import requests
from urllib.request import urlretrieve
from bs4 import BeautifulSoup 
import os 

#current dir
cwd = os.getcwd()

In [ ]:
URL = "https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/kenkou_iryou/shokuhin/yunyu_kanshi/index_00017.html"
BASE = "https://www.mhlw.go.jp"
#current dir
cwd = os.getcwd()
#directroy path to save all downloaded xls 
SAVE_EXCEL_DIR = cwd+"/xls/"
#save directory of json
SAVE_EXCEL_DIR = cwd+"/json/"
# yeard div in html page
year_div = "m-hdgLv3__hdg"

## SubTask 1
For the First task libraries like urlib and BeautifulSoup , so could extract the urls and download teh xls files and save them in folder with name /xls/ with format Year_Month.xls .

In [ ]:
"""""
    Function that check if the excel already was parsed
    
    parameters:
            
            resp: respone of url 
            
            mode: which mode would be follow when downloading xls. 
                0: Download only new xls
                1: redownload all
"""""

def get_urls_list(resp,mode= 0):
    excels = dict() #create a dictionary of years
    
    soup = BeautifulSoup(resp.text)
    year  = soup.find_all("h3", {"class": year_div}) #get all years
    tables = soup.find_all("table")#
    
    #for each table of the year
    for t in  range(0,len(tables)): 
        yearly_xls = dict() #create a dictionary for each year, with  {month:urls of xls} 
        for month in tables[t].find_all("td"):
            if (month.find('a')) and (month.find('a').get('href').endswith('xls')):
                #print("year: ",year[t].text ," , ",month.next_element.strip()  ," : ", month.find('a').get('href'))
                yearly_xls[month.next_element.strip()] = BASE+month.find('a').get('href') 
        
        excels[year[t].text] = yearly_xls # add each dictionary of year in another dictionary
    
    return excels
"""""
    Function that download xls from given urls
    
    parameters:
            
            urls: list off all urls
            
            mode: which mode would be follow when downloading xls. 
                0: Download only new xls, that doesn exits in xls folder
                1: redownload all found xls
"""""

def download_xls(urls, mode):
    for y in urls.items():
        print("Downloading xls for year: "+y[0])
        for m in y[1].items():
            filename = SAVE_EXCEL_DIR+y[0]+'_'+m[0]+'xls'
            if os.path.isfile(filename) and mode == 0:
                print ("File exist. No action would take.")
            else:
                print ("File does not exist or mode is on overwrite.Creating new xls for Month "+m[0])
                resp = requests.get(m[1])
                try:
                    with open(SAVE_EXCEL_DIR+y[0]+'_'+m[0]+'xls', 'wb') as file:
                        file.write(resp.content)
                except IOError:
                    print("Unable to create file on disk.")

                    
                    
if __name__ == "__main__":
    resp = requests.get(URL)
    urls = get_urls_list(resp)
    download_xls(urls,mode= 0)  
    

## SubTask 2
1. For the 2nd Task  the function get_xls checks for the given folder all files and return them.
2.  Function create_json get each file and transform it to json. The json file has the format of the table, with nested items only the row as a list,like example below and save in folder /json/:
    { "YEAR" : 2011,
       "Month" : "Apr",
       "ITEMS" :\[{row1},{row2},... \]
    }

In [ ]:
import pandas as pd
import json

def get_xls(path):
    files = os.listdir(path)
    for f in files:
        print(f)
    return files

def create_json(filename, json_folder = SAVE_JSON_DIR):
    year,month = os.path.splitext(os.path.basename(filename))[0].split('_')
    print(year,month)
    df = pd.read_excel(filename,header = 1)
    # converting the string to datetime format
    #return None
    json_str = df.to_json(orient='records')
    str_json = '{ "YEAR":'+'2011'+','+'"MONTH":"'+'Apr'+'","ITEMS":'+json_str+"}" 
    #print(str_json)
    
    savefilename = year+'_'+month+'.json'
    with open(json_folder+savefilename,'w') as file:
        file.write(str_json)


        
if __name__ == "__main__":
    xlsFiles = get_xls(SAVE_EXCEL_DIR)
    for f in xlsFiles:
        create_json(filename = SAVE_EXCEL_DIR+f)

## Subtask 3
The 3rd Subtask was not implemented, but here the approach that would follow:
 1. clear data from unessary columns like publication date and remarks.
 2. For Feautures extraction in column "CONTENTS OF VIOLOTAION" word embedings like doc2vec
        